In [1]:
import sys
import os
import numpy as np
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [3]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv')
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [4]:
##
# Prepare data for model training:
#
mono_good = data_handling.query_data_by_labels_and_types(data = cell_images, labels = cell_labels, types = cell_types,
    filter_by_labels = 0.0, filter_by_types = 'mono')
poly_good = data_handling.query_data_by_labels_and_types(data = cell_images, labels = cell_labels, types = cell_types,
    filter_by_labels = 0.0, filter_by_types = 'poly')
X_train = np.concatenate((mono_good, poly_good), axis = 0)
X_train = np.stack((X_train,) * 3, axis = -1)

In [6]:
##
# Train classification model (second phase):
#
optimizer = model_development.create_optimizer('adam')
history, training_time = model_development.train_reconstruction_model(2, '../models/weights/CAE_VGG19_1.00.00_fs.hdf5',
    optimizer = optimizer, training_metrics = model_development.ssim_loss, 
    model_name = 'CAE_VGG19', version = '1.00.00_ss',
    X = X_train, Y = X_train, 
    metric_to_monitor = 'val_loss', no_of_epochs = 200, batch_size = 4, validation_split_ratio = 0.2)

Epoch 1/200
302/302 [==============================] - ETA: 0s - loss: 0.2003
Epoch 1: val_loss improved from inf to 0.24748, saving model to ../models/weights\CAE_VGG19_1.00.00_ss.hdf5
302/302 [==============================] - 158s 501ms/step - loss: 0.2003 - val_loss: 0.2475
Epoch 2/200
302/302 [==============================] - ETA: 0s - loss: 0.1932
Epoch 2: val_loss did not improve from 0.24748
302/302 [==============================] - 146s 483ms/step - loss: 0.1932 - val_loss: 0.2488
Epoch 3/200
302/302 [==============================] - ETA: 0s - loss: 0.1892
Epoch 3: val_loss did not improve from 0.24748
302/302 [==============================] - 148s 489ms/step - loss: 0.1892 - val_loss: 0.2512
Epoch 4/200
302/302 [==============================] - ETA: 0s - loss: 0.1862
Epoch 4: val_loss did not improve from 0.24748
302/302 [==============================] - 141s 469ms/step - loss: 0.1862 - val_loss: 0.2477
Epoch 5/200
302/302 [==============================] - ETA: 0s - lo